In [1]:
from tool import *
from openpyxl import Workbook
from openpyxl import load_workbook
from openpyxl.utils import get_column_letter
from openpyxl.drawing.image import Image
from openpyxl.styles import PatternFill, Alignment, Font, Border, Side

In [2]:
stuffing_table = pd.read_clipboard() # read stuffing table from excel

In [5]:
container_simulation = pd.Series({'20FT':0, '40HC':0, '22MR410':0, '24MR400':0, '27MR400':0}) # 초기값 설정

In [40]:
def find_another_container_stuffing(sp_requested, stuffing_table):
    for case in stuffing_table.iterrows():
        if ((sp_requested - (container_simulation + case[1]).dropna()) < 0).sum() > 0: # 특정 모델 1개라도 SP대비 과잉이면 그 컨테이너 조합을 사용불가한 경우이므로 다음 컨테이너 장입 case로 continue
            print('이건 아니잖아!')
            continue
        elif (sp_requesated - (container_simulation + case[1]).dropna()).sum() > 0:
            con_type = case[1]['Type']
            container_simulation[con_type] += 1
            container_simulation.update((container_simulation + case[1]).dropna())
            print('이거 말고 좀 더 필요해!')
            new_case = find_another_container_stuffing(sp_requested, stuffing_table)
            if new_case is not None:
                container_simulation.update(new_case)
                return container_simulation
        else:
            print('드디어 찾았네!')
            con_type = case[1]['Type']
            container_simulation[con_type] += 1
            container_simulation.update((container_simulation + case[1]).dropna())
            return container_simulation

In [57]:
def find_another_container_stuffing(sp_requested, stuffing_table, initial_value):
    containers = [pd.Series({'20FT':0, '40HC':0, '22MR410':0, '24MR400':0, '27MR400':0}) for a in range(stuffing_table.shape[0])]
    for case in stuffing_table.iterrows():
        containers[case[0]].update(initial_value)
        if ((sp_requested - (containers[case[0]] + case[1]).dropna()) < 0).sum() > 0: # 특정 모델 1개라도 SP대비 과잉이면 그 컨테이너 조합을 사용불가한 경우이므로 다음 컨테이너 장입 case로 continue
            print('이건 아니잖아!')
            continue
        elif (sp_requested - (containers[case[0]] + case[1]).dropna()).sum() > 0:
            con_type = case[1]['Type']
            containers[case[0]][con_type] += 1
            containers[case[0]].update((containers[case[0]] + case[1]).dropna())
            print('이거 말고 좀 더 필요해!')
            new_case = find_another_container_stuffing(sp_requested, stuffing_table, containers[case[0]])
            if new_case is not None:
                container_simulation.update(new_case)
                return container_simulation
        else:
            print('드디어 찾았네!')
            con_type = case[1]['Type']
            container_simulation[con_type] += 1
            container_simulation.update((container_simulation + case[1]).dropna())
            return container_simulation

In [63]:
sp = pd.Series({'22MR410':0, '24MR400':1600, '27MR400':512}) # 반영된 SP 값
# sp = pd.Series({'22MR410':840, '24MR400':800})

In [26]:
container_simulation = pd.Series({'20FT':0, '40HC':0, '22MR410':0, '24MR400':0, '27MR400':0}) # 초기값 설정
# find_another_container_stuffing(sp, stuffing_table, container_simulation)